# Neighborhood encoder

test workplace to make the architecture of the Neighborhood encoder

In [1]:
# I use cellpose to run this
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
import time
import glob
import matplotlib.pyplot as plt
import csv
import tifffile
from PIL import Image
import cv2
from natsort import natsorted
import pandas as pd
from skimage.measure import regionprops, label

In [2]:
class neighborhood_encoder(nn.Module):
    def __init__(self, image_size, embedding_size=128):
        super(neighborhood_encoder, self).__init__()

        # Assign the appropriate metrics
        self.image_size = image_size
        self.embedding_size = embedding_size

        # Creates the encoders
        self.image_encoder = self.create_image_encoder()
        self.location_encoder = self.create_location_encoder()
        self.morphology_encoder = self.create_morphology_encoder()

        #
    
    def create_image_encoder(self):
        return nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * self.image_size * self.image_size, self.embedding_size),
            nn.ReLU()
        )
    
    def create_location_encoder(self):
        return nn.Sequential(

        )

    def create_morphology_encoder(self):
        return nn.Sequential(
            
        )
    
    def normalize_by_mask(self, raw_img, mask_img):

        mask = mask_img > 0 

        if mask.sum() == 0:
            print("Warning: No nucleus pixels detected in the mask! Returning raw image.")
            return raw_img.astype(np.float32)

        nucleus_pixels = raw_img[mask]
        mean = nucleus_pixels.mean()
        std = nucleus_pixels.std() if nucleus_pixels.std() > 0 else 1.0

        normalized_img = raw_img.astype(np.float32).copy()
        normalized_img[mask] = (raw_img[mask] - mean) / std

        return normalized_img

    def forward(self, x):
        #fill it in here when done
        return

### Code to load the csv data into tensor format
This code dynamically loads the csv files in the same folder 
1. It creates a 3d tensor where each 2d matrix is a csv file
2. Saves the name of the file as its feature name in a list

In [3]:
path_to_experiment = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\WvN014_phenotype_imaging_strategy5_20250217\_from_Li_Results_ROI9"

if not os.path.exists(path_to_experiment):
    print("path to experiment folder doesn't exist")

path_to_table_data = os.path.join(path_to_experiment, "template_feature_tables")

if not os.path.exists(path_to_experiment):
    print("path to table folder doesn't exist")

csv_files_paths = glob.glob(os.path.join(path_to_table_data, "*.csv"))

tensor_list = []
feature_name_list = []

for csv_file_path in csv_files_paths:

    feature_name = os.path.splitext(os.path.basename(csv_file_path))[0]
    feature_name_list.append(feature_name)
    # print(feature_name_list)

    with open(csv_file_path, newline = '') as csvfile:
        reader = csv.reader(csvfile)
        data = [list(map(float, row)) for row in reader]
        data_tensor = torch.tensor(data, dtype=torch.float32)

        if data_tensor.shape[1] == 1:
            data_tensor = data_tensor.repeat(1, 101)

        tensor_list.append(data_tensor)
        # print(data_tensor.shape)
        # print(len(tensor_list))

feature_index = {name: i for i, name in enumerate(feature_name_list)}
all_cells_tensor = torch.stack(tensor_list, dim=0)
print(all_cells_tensor.shape)
print(feature_name_list)
print(f"There are suppose to be {len(feature_name_list)} feature dimensions in the torch tensor")

torch.Size([6, 547, 101])
['tb_brightness', 'tb_coordinates_x', 'tb_circularity', 'tb_coordinates_y', 'tb_cell_category', 'tb_area']
There are suppose to be 6 feature dimensions in the torch tensor


In [3]:
path_to_experiment = r"\\store\department\gene\chien_data\Lab\Data_and_Analysis\Wilco_van_Nes\WvN014_phenotype_imaging_strategy5_20250217\_from_Li_Results_ROI9"

if not os.path.exists(path_to_experiment):
    print("path to experiment folder doesn't exist")

path_to_table_data = os.path.join(path_to_experiment, "template_feature_tables")

if not os.path.exists(path_to_experiment):
    print("path to table folder doesn't exist")

path_to_raw_feature_data = os.path.join(path_to_table_data, "raw_feature_data_per_ROI")

if not os.path.exists(path_to_raw_feature_data):
    print("path to raw features folder doesn't exist")

csv_files_paths = glob.glob(os.path.join(path_to_raw_feature_data, "*ROI9.csv"))

print(csv_files_paths)

tensor_list = []
feature_name_list = []

for csv_file_path in csv_files_paths:

    feature_name = os.path.splitext(os.path.basename(csv_file_path))[0]
    feature_name_list.append(feature_name)
    # print(feature_name_list)

    with open(csv_file_path, newline = '') as csvfile:
        reader = csv.reader(csvfile)
        data = [list(map(float, row)) for row in reader]
        data_tensor = torch.tensor(data, dtype=torch.float32)

        if data_tensor.shape[1] == 1:
            data_tensor = data_tensor.repeat(1, 101)

        tensor_list.append(data_tensor)
        # print(data_tensor.shape)
        # print(len(tensor_list))

feature_index = {name: i for i, name in enumerate(feature_name_list)}
all_cells_tensor = torch.stack(tensor_list, dim=0)
print(all_cells_tensor.shape)
print(feature_name_list)
print(f"There are suppose to be {len(feature_name_list)} feature dimensions in the torch tensor")

['\\\\store\\department\\gene\\chien_data\\Lab\\Data_and_Analysis\\Wilco_van_Nes\\WvN014_phenotype_imaging_strategy5_20250217\\_from_Li_Results_ROI9\\template_feature_tables\\raw_feature_data_per_ROI\\area_ROI9.csv', '\\\\store\\department\\gene\\chien_data\\Lab\\Data_and_Analysis\\Wilco_van_Nes\\WvN014_phenotype_imaging_strategy5_20250217\\_from_Li_Results_ROI9\\template_feature_tables\\raw_feature_data_per_ROI\\cell_type_ROI9.csv', '\\\\store\\department\\gene\\chien_data\\Lab\\Data_and_Analysis\\Wilco_van_Nes\\WvN014_phenotype_imaging_strategy5_20250217\\_from_Li_Results_ROI9\\template_feature_tables\\raw_feature_data_per_ROI\\circularity_ROI9.csv', '\\\\store\\department\\gene\\chien_data\\Lab\\Data_and_Analysis\\Wilco_van_Nes\\WvN014_phenotype_imaging_strategy5_20250217\\_from_Li_Results_ROI9\\template_feature_tables\\raw_feature_data_per_ROI\\x_ROI9.csv', '\\\\store\\department\\gene\\chien_data\\Lab\\Data_and_Analysis\\Wilco_van_Nes\\WvN014_phenotype_imaging_strategy5_20250217\\

In [4]:
image_size = 32
image_size_half = image_size //2

neighborhood_encoder_model = neighborhood_encoder(image_size)

In [5]:
x_coords = all_cells_tensor[feature_index['x_ROI9']]
y_coords = all_cells_tensor[feature_index['y_ROI9']]
cell_type = all_cells_tensor[feature_index['cell_type_ROI9']]

nucleus_path = os.path.join(path_to_table_data, "637nm")
nucleus_mask_path = os.path.join(path_to_table_data, "637nm_ImgGS")

nucleus_files = natsorted(glob.glob(os.path.join(nucleus_path, "*.tif")))
nucleus_mask_files = natsorted(glob.glob(os.path.join(nucleus_mask_path, "*.tif")))

print(f"Found {len(nucleus_files)} nucleus files.")
print(f"Found {len(nucleus_mask_files)} nucleus mask files.")

if len(nucleus_files) != len(nucleus_mask_files):
    print("Warning: nucleus files and mask files don't match in length!")
else:
    print(f"Both lists have the same length: {len(nucleus_files)}")

output_dir = os.path.join(path_to_table_data, "cropped_cells")
output_dir_637nm = os.path.join(output_dir, "637nm")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(output_dir_637nm):
    os.makedirs(output_dir_637nm)

for frame_idx, (nucleus_file, nucleus_mask_file) in enumerate(zip(nucleus_files, nucleus_mask_files)):

    nucleus_img = cv2.imread(nucleus_file, -1)
    nucleus_mask_img = cv2.imread(nucleus_mask_file, -1)

    nucleus_img_padded = np.pad(nucleus_img, image_size_half, mode='constant', constant_values=0)
    nucleus_mask_img_padded = np.pad(nucleus_mask_img, image_size_half, mode='constant', constant_values=0)

    for cell_id in range(all_cells_tensor.shape[1]):
        
        if cell_type[cell_id, frame_idx] == 0:
            continue
        
        x = int(x_coords[cell_id, frame_idx].item())
        y = int(y_coords[cell_id, frame_idx].item())
        x_padded = x + image_size_half
        y_padded = y + image_size_half

        crop_img = nucleus_img_padded[y_padded - image_size_half:y_padded + image_size_half,
                              x_padded - image_size_half:x_padded + image_size_half]

        crop_mask = nucleus_mask_img_padded[y_padded - image_size_half:y_padded + image_size_half,
                                    x_padded - image_size_half:x_padded + image_size_half]
        
        crop_img_norm = neighborhood_encoder_model.normalize_by_mask(crop_img, crop_mask)

        output_filename = f"cell_{cell_id}_frame{frame_idx}_ROI9.tif"
        output_path = os.path.join(output_dir_637nm, output_filename)
        cv2.imwrite(output_path, crop_img_norm)

Found 101 nucleus files.
Found 101 nucleus mask files.
Both lists have the same length: 101


In [46]:
import re

cropped_img_dir = os.path.join(path_to_table_data, "cropped_cells", "637nm")

image_paths = natsorted(glob.glob(os.path.join(cropped_img_dir, "*.tif")))

num_cells = all_cells_tensor.shape[1]
num_frames = all_cells_tensor.shape[2]
image_size = 32

image_tensor_map = torch.zeros((num_cells, num_frames, 1, image_size, image_size), dtype=torch.float32)

pattern = re.compile(r"cell_(\d+)_frame(\d+)_ROI9\.tif")

missing_images = []

for path in image_paths:
    filename = os.path.basename(path)
    match = pattern.match(filename)
    if match:
        cell_id = int(match.group(1))
        frame_idx = int(match.group(2))

        if cell_id >= num_cells or frame_idx >= num_frames:
            print(f"Skipping out-of-bounds image: {filename}")
            continue

        img = cv2.imread(path, cv2.IMREAD_UNCHANGED).astype(np.float32)
        img_tensor = torch.from_numpy(img).unsqueeze(0)

        image_tensor_map[cell_id, frame_idx] = img_tensor
    else:
        missing_images.append(filename)

print(f"Loaded image tensor map of shape: {image_tensor_map.shape}")
if missing_images:
    print(f"Warning: Could not parse {len(missing_images)} filenames.")

RuntimeError: The expanded size of the tensor (32) must match the existing size (31) at non-singleton dimension 2.  Target sizes: [1, 32, 32].  Tensor sizes: [32, 31]

### Code to extract and save the relevant data from the masks

In [ ]:
# Loading the images
nucleus_path = os.path.join(path_to_table_data, "637nm")
nucleus_mask_path = os.path.join(path_to_table_data, "637nm_ImgGS")
WL_mask_path = os.path.join(path_to_table_data, "WhiteLight_ImgGS")

nucleus_files = natsorted(glob.glob(os.path.join(nucleus_path, "*.tif")))
nucleus_mask_files = natsorted(glob.glob(os.path.join(nucleus_mask_path, "*.tif")))
WL_mask_files = natsorted(glob.glob(os.path.join(WL_mask_path, "*.tif")))

print(f"Found {len(nucleus_files)} nucleus files.")
print(f"Found {len(nucleus_mask_files)} nucleus mask files.")
print(f"Found {len(WL_mask_files)} WhiteLight mask files.")

if len(nucleus_files) != len(nucleus_mask_files):
    print("Warning: nucleus files and mask files don't match in length!")
else:
    print(f"Both lists have the same length: {len(nucleus_files)}")
    
if len(WL_mask_files) != len(nucleus_mask_files):
    print("Warning: WhiteLight mask files and nucleus mask files don't match in length!")
else:
    print(f"Both WhiteLight mask files and nucleus mask files have the same length: {len(WL_mask_files)}")

output_table_data_path = os.path.join(path_to_table_data, "raw_feature_data_per_ROI")

if not os.path.exists(output_table_data_path):
    os.makedirs(output_table_data_path)

x_list, y_list, area_list, circ_list, eccentricity_list, cell_type_list = [], [], [], [], [], []

max_cells = 0
ROI = 9
total_frames = len(nucleus_mask_files)

for frame_idx, (nucleus_file, nucleus_mask_file, WL_mask_file) in enumerate(zip(nucleus_files, nucleus_mask_files, WL_mask_files)):

    nucleus_img = np.array(Image.open(nucleus_file).convert("I;16"))
    nucleus_mask_img = np.array(Image.open(nucleus_mask_file).convert("I;16"))
    WL_mask_img = np.array(Image.open(WL_mask_file).convert("I;16"))

    labeled_mask_nucleus = label(nucleus_mask_img)
    labeled_mask_WL = label(WL_mask_img)

    props_nucleus = regionprops(labeled_mask_nucleus, intensity_image=nucleus_img)
    props_WL = regionprops(labeled_mask_WL, intensity_image=WL_mask_img)

    print(f"Frame {frame_idx}: {len(props_nucleus)} cells in nucleus mask, {len(props_WL)} cells in WL mask")

    x_row, y_row, area_row, circ_row, eccentricity_row, cell_type_row = [], [], [], [], [], []

    nucleus_props_by_pos = {
        (int(prop.centroid[0]), int(prop.centroid[1])): prop
        for prop in props_nucleus
    }

    for region_idx, prop_WL in enumerate(props_WL):
        WL_coords = prop_WL.coords
        WL_label_img = labeled_mask_WL == prop_WL.label

        matched = False
        for (y_nuc, x_nuc), nuc_prop in nucleus_props_by_pos.items():
            if WL_label_img[y_nuc, x_nuc]:
                y, x = y_nuc, x_nuc
                cell_type_row.append(1)
                matched = True
                break

        if not matched:
            y, x = map(int, prop_WL.centroid)
            cell_type_row.append(2)

        area = prop_WL.area
        perimeter = prop_WL.perimeter if prop_WL.perimeter > 0 else 1
        circ = 4 * np.pi * (area / (perimeter ** 2))
        eccentricity = prop_WL.eccentricity

        x_row.append(x)
        y_row.append(y)
        area_row.append(area)
        circ_row.append(circ)
        eccentricity_row.append(eccentricity)

    max_cells = max(max_cells, len(x_row))

    x_list.append(x_row)
    y_list.append(y_row)
    area_list.append(area_row)
    circ_list.append(circ_row)
    eccentricity_list.append(eccentricity_row)
    cell_type_list.append(cell_type_row)

def pad_feature_matrix(feature_lists, max_cells):
    num_frames = len(feature_lists)
    feature_matrix = np.zeros((max_cells, num_frames), dtype=np.float32)
    for frame_idx, row in enumerate(feature_lists):
        feature_matrix[:len(row), frame_idx] = row
    return feature_matrix

features = {
    f"x_ROI{ROI}.csv": pad_feature_matrix(x_list, max_cells),
    f"y_ROI{ROI}.csv": pad_feature_matrix(y_list, max_cells),
    f"area_ROI{ROI}.csv": pad_feature_matrix(area_list, max_cells),
    f"circularity_ROI{ROI}.csv": pad_feature_matrix(circ_list, max_cells),
    f"eccentricity_ROI{ROI}.csv": pad_feature_matrix(eccentricity_list, max_cells),
    f"cell_type_ROI{ROI}.csv": pad_feature_matrix(cell_type_list, max_cells),
}

for filename, matrix in features.items():
    df = pd.DataFrame(matrix)
    df.to_csv(os.path.join(output_table_data_path, filename), index=False, header=False)

Found 101 nucleus files.
Found 101 nucleus mask files.
Found 101 WhiteLight mask files.
Both lists have the same length: 101
Both WhiteLight mask files and nucleus mask files have the same length: 101


In [ ]:
# Loading the images
nucleus_path = os.path.join(path_to_table_data, "637nm")
nucleus_mask_path = os.path.join(path_to_table_data, "637nm_ImgGS")
WL_mask_path = os.path.join(path_to_table_data, "WhiteLight_ImgGS")

nucleus_files = natsorted(glob.glob(os.path.join(nucleus_path, "*.tif")))
nucleus_mask_files = natsorted(glob.glob(os.path.join(nucleus_mask_path, "*.tif")))
WL_mask_files = natsorted(glob.glob(os.path.join(WL_mask_path, "*.tif")))

print(f"Found {len(nucleus_files)} nucleus files.")
print(f"Found {len(nucleus_mask_files)} nucleus mask files.")
print(f"Found {len(WL_mask_files)} WhiteLight mask files.")

if len(nucleus_files) != len(nucleus_mask_files):
    print("Warning: nucleus files and mask files don't match in length!")
else:
    print(f"Both lists have the same length: {len(nucleus_files)}")
    
if len(WL_mask_files) != len(nucleus_mask_files):
    print("Warning: WhiteLight mask files and nucleus mask files don't match in length!")
else:
    print(f"Both WhiteLight mask files and nucleus mask files have the same length: {len(WL_mask_files)}")

output_table_data_path = os.path.join(path_to_table_data, "raw_feature_data_per_ROI")

if not os.path.exists(output_table_data_path):
    os.makedirs(output_table_data_path)

x_list, y_list, area_list, circ_list, eccentricity_list, cell_type_list = [], [], [], [], [], []

max_cells = 0
ROI = 9
total_frames = len(nucleus_mask_files)

for frame_idx, (nucleus_file, nucleus_mask_file, WL_mask_file) in enumerate(zip(nucleus_files, nucleus_mask_files, WL_mask_files)):

    nucleus_img = np.array(Image.open(nucleus_file).convert("I;16"))
    nucleus_mask_img = np.array(Image.open(nucleus_mask_file).convert("I;16"))
    WL_mask_img = np.array(Image.open(WL_mask_file).convert("I;16"))

    labeled_mask_nucleus = label(nucleus_mask_img)
    labeled_mask_WL = label(WL_mask_img)

    props_nucleus = regionprops(labeled_mask_nucleus, intensity_image=nucleus_img)
    props_WL = regionprops(labeled_mask_WL, intensity_image=WL_mask_img)

    print(f"Frame {frame_idx}: {len(props_nucleus)} cells in nucleus mask, {len(props_WL)} cells in WL mask")

    for prop in props_nucleus:

        WL_coords = prop_WL.coords
        WL_label_img = labeled_mask_WL == prop_WL.label

    if max(len(props_nucleus), len(props_WL)) > max_cells:
        max_cells = max(len(props_nucleus), len(props_WL))

Found 101 nucleus files.
Found 101 nucleus mask files.
Found 101 WhiteLight mask files.
Both lists have the same length: 101
Both WhiteLight mask files and nucleus mask files have the same length: 101
Frame 0: 405 cells in nucleus mask, 352 cells in WL mask
Frame 1: 407 cells in nucleus mask, 368 cells in WL mask
Frame 2: 405 cells in nucleus mask, 399 cells in WL mask
Frame 3: 412 cells in nucleus mask, 398 cells in WL mask
Frame 4: 421 cells in nucleus mask, 356 cells in WL mask
Frame 5: 427 cells in nucleus mask, 390 cells in WL mask
Frame 6: 426 cells in nucleus mask, 366 cells in WL mask
Frame 7: 427 cells in nucleus mask, 365 cells in WL mask
Frame 8: 427 cells in nucleus mask, 367 cells in WL mask
Frame 9: 422 cells in nucleus mask, 390 cells in WL mask
Frame 10: 420 cells in nucleus mask, 382 cells in WL mask
Frame 11: 420 cells in nucleus mask, 363 cells in WL mask
Frame 12: 416 cells in nucleus mask, 368 cells in WL mask
Frame 13: 416 cells in nucleus mask, 342 cells in WL ma

In [14]:
print(max_cells)

495
